
<h1><center>Capstone Project</center></h1>

In [1]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [2]:
# Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/Parishes_of_Jamaica, 
#in order to obtain the data that is in the table of parishes and to transform the data into a pandas dataframe

url='https://en.wikipedia.org/wiki/Parishes_of_Jamaica'

df=pd.read_html(url, header=0)[2]

df.head()

,Parish,Parish.1,Capital,Areakm²,PopulationCensus 2011[1]
0,Cornwall County,Cornwall County,Cornwall County,Cornwall County,Cornwall County
1,1,Hanover,Lucea,450.4,69533
2,2,Saint Elizabeth,Black River,1212.4,150205
3,3,Saint James,Montego Bay,594.9,183811
4,4,Trelawny,Falmouth,874.6,75164


In [3]:
## Remove Unnecessary columns from data frame
df = df.drop(['Parish','Areakm²','PopulationCensus 2011[1]'],axis=1)


In [4]:
## Remove counties from dataframe
df = df.drop([0,6,12,17],axis=0)


In [9]:
## Rename the parish column 
df = df.rename(columns = {'Parish.1':'Parish'})
df

,Parish,Capital
1,Hanover,Lucea
2,Saint Elizabeth,Black River
3,Saint James,Montego Bay
4,Trelawny,Falmouth
5,Westmoreland,Savanna-la-Mar
7,Clarendon,May Pen
8,Manchester,Mandeville
9,Saint Ann,Saint Ann's Bay
10,Saint Catherine,Spanish Town
11,Saint Mary,Port Maria


In [10]:
## Remove subscripts from data rows 13 and 15

df.Parish.loc[df.Parish == 'Kingston Parish(1)(2)'] = 'Kingston Parish'
df

,Parish,Capital
1,Hanover,Lucea
2,Saint Elizabeth,Black River
3,Saint James,Montego Bay
4,Trelawny,Falmouth
5,Westmoreland,Savanna-la-Mar
7,Clarendon,May Pen
8,Manchester,Mandeville
9,Saint Ann,Saint Ann's Bay
10,Saint Catherine,Spanish Town
11,Saint Mary,Port Maria


In [11]:
df.Parish.loc[df.Parish == 'Saint Andrew(1)'] = 'Saint Andrew'
df

,Parish,Capital
1,Hanover,Lucea
2,Saint Elizabeth,Black River
3,Saint James,Montego Bay
4,Trelawny,Falmouth
5,Westmoreland,Savanna-la-Mar
7,Clarendon,May Pen
8,Manchester,Mandeville
9,Saint Ann,Saint Ann's Bay
10,Saint Catherine,Spanish Town
11,Saint Mary,Port Maria


In [12]:
# Sort and Reset index.
df=df.sort_values(by=['Parish'], ascending=[1]).reset_index(drop=True)

In [13]:
df.head()

,Parish,Capital
0,Clarendon,May Pen
1,Hanover,Lucea
2,Kingston Parish,Kingston
3,Manchester,Mandeville
4,Portland,Port Antonio


In [14]:
#Use the .shape method to print the number of rows of your dataframe.
df.shape

(14, 2)

In [15]:
## Get the coordinated for each parish in Jamaica

addresses = []
latitudes = []
longitudes = []
for index, row in df.iterrows():
    address = row['Capital'] +', '+row['Parish']
    addresses.append(address)
    
    geolocator = Nominatim(user_agent="to_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    latitudes.append(latitude)
    longitudes.append(longitude)
    print('{}: Latitude= {}, Longitude= {}.'.format(address,latitude,longitude))

May Pen, Clarendon: Latitude= 17.974637, Longitude= -77.2581951.
Lucea, Hanover: Latitude= 18.442328, Longitude= -78.1726839.
Kingston, Kingston Parish: Latitude= 17.9712148, Longitude= -76.7928128.
Mandeville, Manchester: Latitude= 18.0431254, Longitude= -77.50741.
Port Antonio, Portland: Latitude= 18.1763606, Longitude= -76.4459288.
Half Way Tree, Saint Andrew: Latitude= 18.0118757, Longitude= -76.7968643.
Saint Ann's Bay, Saint Ann: Latitude= 18.435614, Longitude= -77.2023419.
Spanish Town, Saint Catherine: Latitude= 17.9956926, Longitude= -76.9540831.
Black River, Saint Elizabeth: Latitude= 18.0269811, Longitude= -77.8482147.
Montego Bay, Saint James: Latitude= 18.4724603, Longitude= -77.9217357.
Port Maria, Saint Mary: Latitude= 18.3707461, Longitude= -76.8915489.
Morant Bay, Saint Thomas: Latitude= 17.8820832, Longitude= -76.4115503.
Falmouth, Trelawny: Latitude= 18.4929078, Longitude= -77.6574376.
Savanna-la-Mar, Westmoreland: Latitude= 18.2193686, Longitude= -78.1326673.


In [16]:
## Add coordinates to dataframe

df['Latitude'] = latitudes
df['Longitude'] = longitudes
df

,Parish,Capital,Latitude,Longitude
0,Clarendon,May Pen,17.974637,-77.258195
1,Hanover,Lucea,18.442328,-78.172684
2,Kingston Parish,Kingston,17.971215,-76.792813
3,Manchester,Mandeville,18.043125,-77.507410
4,Portland,Port Antonio,18.176361,-76.445929
5,Saint Andrew,Half Way Tree,18.011876,-76.796864
6,Saint Ann,Saint Ann's Bay,18.435614,-77.202342
7,Saint Catherine,Spanish Town,17.995693,-76.954083
8,Saint Elizabeth,Black River,18.026981,-77.848215
9,Saint James,Montego Bay,18.472460,-77.921736


<p>Create a map of  with parishes superimposed on top.</P>

In [17]:
# create map of using latitude and longitude values
map_jamaica = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng,capital, parish in zip(df['Latitude'], df['Longitude'],df['Capital'], df['Parish']):
    label = '{}, {}'.format(capital,parish)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_jamaica)
    
map_jamaica

We are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'ARRTY55QVTY1GJYBTKDQGKLUL5TOO5SEYQU3MWG2QRDCFHY1' # your Foursquare ID
CLIENT_SECRET = 'RTWDQMILMFBKBLJCMASS32DADGO13BOYMDF5O4CYUH4MA0GB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 35
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ARRTY55QVTY1GJYBTKDQGKLUL5TOO5SEYQU3MWG2QRDCFHY1
CLIENT_SECRET:RTWDQMILMFBKBLJCMASS32DADGO13BOYMDF5O4CYUH4MA0GB


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=15000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
jamaican_venues = getNearbyVenues(names=df.Capital,
                                   latitudes=df.Latitude,
                                   longitudes=df.Longitude
                                  )


May Pen
Lucea
Kingston
Mandeville
Port Antonio
Half Way Tree
Saint Ann's Bay
Spanish Town
Black River
Montego Bay
Port Maria
Morant Bay
Falmouth
Savanna-la-Mar


In [21]:
print(jamaican_venues.shape)
jamaican_venues

(301, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,May Pen,17.974637,-77.258195,Homar's Roti & The Grill,17.955390,-77.232362,Indian Restaurant
1,May Pen,17.974637,-77.258195,Murray's Fish and Jerk Hut,17.990426,-77.368564,Caribbean Restaurant
2,May Pen,17.974637,-77.258195,Juici Patties,17.995355,-77.374100,Fast Food Restaurant
3,May Pen,17.974637,-77.258195,Fyah Side,17.969874,-77.352091,BBQ Joint
4,May Pen,17.974637,-77.258195,Juici Patties,17.970148,-77.245048,Fast Food Restaurant
5,May Pen,17.974637,-77.258195,Payless Shoesource,17.966802,-77.242359,Shoe Store
6,May Pen,17.974637,-77.258195,Burger King,17.967499,-77.241759,Fast Food Restaurant
7,May Pen,17.974637,-77.258195,Rio Minho,17.956346,-77.256346,River
8,Lucea,18.442328,-78.172684,Grand Palladium Lady Hamilton Resort & Spa,18.457286,-78.152699,Resort
9,Lucea,18.442328,-78.172684,Grand Palladium Jamaica Resort & Spa,18.458698,-78.150561,Resort


In [22]:
## Identifying and selecting only those venues unter a category related to speciality restaurants

jamaican_venues['Venue Category'].unique()

jamaican_food_venues = jamaican_venues.loc[jamaican_venues['Venue Category'].isin(['Indian Restaurant', 'Caribbean Restaurant',
        'Steakhouse', 'Seafood Restaurant', 'Italian Restaurant', 
        'Sushi Restaurant', 
        'Cajun / Creole Restaurant',
         'Chinese Restaurant',
         'Japanese Restaurant',
       'Mexican Restaurant',  
       'Asian Restaurant', 'American Restaurant',  'French Restaurant'])]
jamaican_food_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,May Pen,17.974637,-77.258195,Homar's Roti & The Grill,17.955390,-77.232362,Indian Restaurant
1,May Pen,17.974637,-77.258195,Murray's Fish and Jerk Hut,17.990426,-77.368564,Caribbean Restaurant
16,Lucea,18.442328,-78.172684,Bubba's Steakhouse,18.457883,-78.151802,Steakhouse
17,Lucea,18.442328,-78.172684,Poseydon's Beach Restaurant,18.454892,-78.152994,Seafood Restaurant
19,Lucea,18.442328,-78.172684,Hammond's Bakery,18.443290,-78.170908,Caribbean Restaurant
21,Lucea,18.442328,-78.172684,arte and cuccina,18.457962,-78.151721,Italian Restaurant
41,Kingston,17.971215,-76.792813,East Japanese Restaurant,18.022242,-76.797781,Sushi Restaurant
51,Kingston,17.971215,-76.792813,Triple T'z Eatery,18.023759,-76.791755,Cajun / Creole Restaurant
58,Kingston,17.971215,-76.792813,Nirvanna Indian Fusion Cuisine,18.017027,-76.781062,Indian Restaurant
59,Kingston,17.971215,-76.792813,Saffron Indian Cuisine,18.022214,-76.797716,Indian Restaurant


#### Count of venues returned for each Parish

In [24]:
jamaican_food_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Black River,1,1,1,1,1,1
Falmouth,8,8,8,8,8,8
Half Way Tree,6,6,6,6,6,6
Kingston,4,4,4,4,4,4
Lucea,4,4,4,4,4,4
Mandeville,3,3,3,3,3,3
May Pen,2,2,2,2,2,2
Montego Bay,11,11,11,11,11,11
Port Antonio,2,2,2,2,2,2


## 2. Analyze Each Neighborhood

In [25]:
# one hot encoding
jamaica_onehot = pd.get_dummies(jamaican_food_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
jamaica_onehot['Neighbourhood'] = jamaican_food_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [jamaica_onehot.columns[-1]] + list(jamaica_onehot.columns[:-1])
jamaica_onehot = jamaica_onehot[fixed_columns]

jamaica_onehot.head()

,Neighbourhood,American Restaurant,Asian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,French Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Seafood Restaurant,Steakhouse,Sushi Restaurant
0,May Pen,0,0,0,0,0,0,1,0,0,0,0,0,0
1,May Pen,0,0,0,1,0,0,0,0,0,0,0,0,0
16,Lucea,0,0,0,0,0,0,0,0,0,0,0,1,0
17,Lucea,0,0,0,0,0,0,0,0,0,0,1,0,0
19,Lucea,0,0,0,1,0,0,0,0,0,0,0,0,0


#### And let's examine the new dataframe size.

In [26]:
jamaica_onehot.shape

(61, 14)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
jamaica_grouped = jamaica_onehot.groupby('Neighbourhood').mean().reset_index()
jamaica_grouped

,Neighbourhood,American Restaurant,Asian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,French Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Seafood Restaurant,Steakhouse,Sushi Restaurant
0,Black River,0.000000,0.000000,0.000000,1.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Falmouth,0.000000,0.000000,0.000000,0.375000,0.000000,0.00,0.000000,0.125,0.250000,0.000000,0.125000,0.125000,0.000000
2,Half Way Tree,0.000000,0.000000,0.166667,0.000000,0.000000,0.00,0.500000,0.000,0.000000,0.166667,0.000000,0.000000,0.166667
3,Kingston,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.500000,0.000,0.000000,0.000000,0.000000,0.000000,0.250000
4,Lucea,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.250,0.000000,0.000000,0.250000,0.250000,0.000000
5,Mandeville,0.000000,0.000000,0.000000,0.333333,0.666667,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
6,May Pen,0.000000,0.000000,0.000000,0.500000,0.000000,0.00,0.500000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Montego Bay,0.090909,0.090909,0.000000,0.454545,0.090909,0.00,0.090909,0.000,0.000000,0.000000,0.181818,0.000000,0.000000
8,Port Antonio,0.000000,0.000000,0.000000,0.500000,0.000000,0.00,0.000000,0.000,0.500000,0.000000,0.000000,0.000000,0.000000
9,Port Maria,0.000000,0.000000,0.000000,0.500000,0.000000,0.25,0.000000,0.250,0.000000,0.000000,0.000000,0.000000,0.000000


#### Let's confirm the new size

In [28]:
jamaica_grouped.shape

(12, 14)

#### Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in jamaica_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = jamaica_grouped[jamaica_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Black River----
                       venue  freq
0       Caribbean Restaurant   1.0
1        American Restaurant   0.0
2           Asian Restaurant   0.0
3  Cajun / Creole Restaurant   0.0
4         Chinese Restaurant   0.0


----Falmouth----
                  venue  freq
0  Caribbean Restaurant  0.38
1   Japanese Restaurant  0.25
2    Italian Restaurant  0.12
3    Seafood Restaurant  0.12
4            Steakhouse  0.12


----Half Way Tree----
                       venue  freq
0          Indian Restaurant  0.50
1  Cajun / Creole Restaurant  0.17
2         Mexican Restaurant  0.17
3           Sushi Restaurant  0.17
4        American Restaurant  0.00


----Kingston----
                       venue  freq
0          Indian Restaurant  0.50
1  Cajun / Creole Restaurant  0.25
2           Sushi Restaurant  0.25
3        American Restaurant  0.00
4           Asian Restaurant  0.00


----Lucea----
                  venue  freq
0  Caribbean Restaurant  0.25
1    Italian Restaurant  0.25
2 

#### Let's put that into a pandas dataframe

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [97]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = jamaica_grouped['Neighbourhood']

for ind in np.arange(jamaica_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(jamaica_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Black River,Caribbean Restaurant,Sushi Restaurant,Steakhouse,Seafood Restaurant,Mexican Restaurant
1,Falmouth,Caribbean Restaurant,Japanese Restaurant,Steakhouse,Seafood Restaurant,Italian Restaurant
2,Half Way Tree,Indian Restaurant,Sushi Restaurant,Mexican Restaurant,Cajun / Creole Restaurant,Steakhouse
3,Kingston,Indian Restaurant,Sushi Restaurant,Cajun / Creole Restaurant,Steakhouse,Seafood Restaurant
4,Lucea,Steakhouse,Seafood Restaurant,Italian Restaurant,Caribbean Restaurant,Sushi Restaurant


## 3. Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters.

In [98]:
# set number of clusters
kclusters = 3

jamaica_grouped_clustering = jamaica_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(jamaica_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 1, 2, 2, 1, 2, 0, 0], dtype=int32)

In [99]:
df_merged = df
df_merged = df_merged.rename(columns = {'Capital':'Neighbourhood'})
df_merged = df_merged.drop([11,13],axis=0)
df_merged

,Parish,Neighbourhood,Latitude,Longitude
0,Clarendon,May Pen,17.974637,-77.258195
1,Hanover,Lucea,18.442328,-78.172684
2,Kingston Parish,Kingston,17.971215,-76.792813
3,Manchester,Mandeville,18.043125,-77.507410
4,Portland,Port Antonio,18.176361,-76.445929
5,Saint Andrew,Half Way Tree,18.011876,-76.796864
6,Saint Ann,Saint Ann's Bay,18.435614,-77.202342
7,Saint Catherine,Spanish Town,17.995693,-76.954083
8,Saint Elizabeth,Black River,18.026981,-77.848215
9,Saint James,Montego Bay,18.472460,-77.921736


In [100]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [101]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

df_merged # check the last columns!

,Parish,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Clarendon,May Pen,17.974637,-77.258195,1,Indian Restaurant,Caribbean Restaurant,Sushi Restaurant,Steakhouse,Seafood Restaurant
1,Hanover,Lucea,18.442328,-78.172684,2,Steakhouse,Seafood Restaurant,Italian Restaurant,Caribbean Restaurant,Sushi Restaurant
2,Kingston Parish,Kingston,17.971215,-76.792813,1,Indian Restaurant,Sushi Restaurant,Cajun / Creole Restaurant,Steakhouse,Seafood Restaurant
3,Manchester,Mandeville,18.043125,-77.507410,2,Chinese Restaurant,Caribbean Restaurant,Sushi Restaurant,Steakhouse,Seafood Restaurant
4,Portland,Port Antonio,18.176361,-76.445929,0,Japanese Restaurant,Caribbean Restaurant,Sushi Restaurant,Steakhouse,Seafood Restaurant
5,Saint Andrew,Half Way Tree,18.011876,-76.796864,1,Indian Restaurant,Sushi Restaurant,Mexican Restaurant,Cajun / Creole Restaurant,Steakhouse
6,Saint Ann,Saint Ann's Bay,18.435614,-77.202342,0,Caribbean Restaurant,Steakhouse,Seafood Restaurant,Japanese Restaurant,Sushi Restaurant
7,Saint Catherine,Spanish Town,17.995693,-76.954083,2,Seafood Restaurant,Caribbean Restaurant,Chinese Restaurant,Sushi Restaurant,Steakhouse
8,Saint Elizabeth,Black River,18.026981,-77.848215,0,Caribbean Restaurant,Sushi Restaurant,Steakhouse,Seafood Restaurant,Mexican Restaurant
9,Saint James,Montego Bay,18.472460,-77.921736,2,Caribbean Restaurant,Seafood Restaurant,Indian Restaurant,Chinese Restaurant,Asian Restaurant


#### Finally, let's visualize the resulting clusters

In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters